Prototype code to randomly sample pointcloud from a CAD file belonging to Shapenet. 
https://towardsdatascience.com/deep-learning-on-point-clouds-implementing-pointnet-in-google-colab-1fd65cd3a263 played an elemental role in understanding how to convert CAD files into pointclouds!

In [ ]:
import numpy as np
import random
import math
!pip install path.py;
from path import Path
import plotly.graph_objects as go
import os

In [ ]:
!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
!unzip -q ModelNet10.zip

path = Path("ModelNet10")

In [ ]:
# To prototype idea on a single image.
# ! wget http://shapenet.cs.stanford.edu/shapenet/obj-zip/ShapeNetCore.v1/02958343/4489a777dd90ebcce28605e174815eef/model.obj 
! wget http://shapenet.cs.stanford.edu/shapenet/obj-zip/ShapeNetCore.v1/03001627/c41fe0605cfe70571c25d54737ed5c8e/model.obj 

In [53]:
# Code forked from https://github.com/thethoughtfulgeek/bpy-visualization-utils/blob/master/mesh.py

def write_off(file, vertices, faces):
    """
    Writes the given vertices and faces to OFF.
    :param vertices: vertices as tuples of (x, y, z) coordinates
    :type vertices: [(float)]
    :param faces: faces as tuples of (num_vertices, vertex_id_1, vertex_id_2, ...)
    :type faces: [(int)]
    """

    num_vertices = len(vertices)
    num_faces = len(faces)

    assert num_vertices > 0
    assert num_faces > 0

    with open(file, 'w') as fp:
        fp.write('OFF\n')
        fp.write(str(num_vertices) + ' ' + str(num_faces) + ' 0\n')

        for vertex in vertices:
            assert len(vertex) == 3, 'invalid vertex with %d dimensions found (%s)' % (len(vertex), file)
            fp.write(str(vertex[0]) + ' ' + str(vertex[1]) + ' ' + str(vertex[2]) + '\n')

        for face in faces:
            assert face[0] == 3, 'only triangular faces supported (%s)' % file
            assert len(face) == 4, 'faces need to have 3 vertices, but found %d (%s)' % (len(face), file)

            for i in range(len(face)):
                assert face[i] >= 0 and face[i] < num_vertices, 'invalid vertex index %d (of %d vertices) (%s)' % (face[i], num_vertices, file)

                fp.write(str(face[i]))
                if i < len(face) - 1:
                    fp.write(' ')

            fp.write('\n')

        # add empty line to be sure
        fp.write('\n')

def read_obj(file):
    """
    Reads vertices and faces from an obj file.
    :param file: path to file to read
    :type file: str
    :return: vertices and faces as lists of tuples
    :rtype: [(float)], [(int)]
    """

    assert os.path.exists(file), 'file %s not found' % file

    with open(file, 'r') as fp:
        lines = fp.readlines()
        lines = [line.strip() for line in lines if line.strip()]

        vertices = []
        faces = []
        for line in lines:
            parts = line.split(' ')
            parts = [part.strip() for part in parts if part]
            if parts[0] == 'v':
                assert len(parts) == 4, \
                    'vertex should be of the form v x y z, but found %d parts instead (%s)' % (len(parts), file)
                assert parts[1] != '', 'vertex x coordinate is empty (%s)' % file
                assert parts[2] != '', 'vertex y coordinate is empty (%s)' % file
                assert parts[3] != '', 'vertex z coordinate is empty (%s)' % file

                vertices.append([float(parts[1]), float(parts[2]), float(parts[3])])
            elif parts[0] == 'f':
                assert len(parts) == 4, \
                    'face should be of the form f v1/vt1/vn1 v2/vt2/vn2 v2/vt2/vn2, but found %d parts (%s) instead (%s)' % (len(parts), line, file)

                components = parts[1].split('/')
                assert len(components) >= 1 and len(components) <= 3, \
                   'face component should have the forms v, v/vt or v/vt/vn, but found %d components instead (%s)' % (len(components), file)
                assert components[0].strip() != '', \
                    'face component is empty (%s)' % file
                v1 = int(components[0])

                components = parts[2].split('/')
                assert len(components) >= 1 and len(components) <= 3, \
                    'face component should have the forms v, v/vt or v/vt/vn, but found %d components instead (%s)' % (len(components), file)
                assert components[0].strip() != '', \
                    'face component is empty (%s)' % file
                v2 = int(components[0])

                components = parts[3].split('/')
                assert len(components) >= 1 and len(components) <= 3, \
                    'face component should have the forms v, v/vt or v/vt/vn, but found %d components instead (%s)' % (len(components), file)
                assert components[0].strip() != '', \
                    'face component is empty (%s)' % file
                v3 = int(components[0])

                #assert v1 != v2 and v2 != v3 and v3 != v2, 'degenerate face detected: %d %d %d (%s)' % (v1, v2, v3, file)
                if v1 == v2 or v2 == v3 or v1 == v3:
                    # print('[Info] skipping degenerate face in %s' % file)
                    continue
                else:
                    faces.append([v1 - 1, v2 - 1, v3 - 1]) # indices are 1-based!
            else:
              # continue to next line until one of the required args is found.
              continue
        return np.array(vertices, dtype=float), np.array(faces, dtype=int)

    assert False, 'could not open %s' % file

In [54]:
# Convert obj to an off file and save locally
def convert_obj_to_off(obj_file, output_filename):
  if not os.path.exists(obj_file):
    print(obj_file, ' does not exist.')
    return
  obj_vertices, obj_faces = read_obj(obj_file)
  assert obj_vertices.shape[1] == 3
  assert obj_faces.shape[1] == 3
  temp_faces = np.ones((obj_faces.shape[0], 4), dtype = int)*3
  # print(obj_faces.shape)
  temp_faces[:, 1:4] = obj_faces[:, :]
  write_off(output_filename, obj_vertices.tolist(), temp_faces.tolist())

In [55]:
def read_cad_off(filename):
  # Read CAD files with .off extension.
  with open(filename, 'r') as f:
    if 'OFF' != f.readline().strip():
      print(f'Error: {filename} is a not a file.')
      return false
    num_vertices, num_faces, _ = tuple([int(s) for s in f.readline().strip().split(' ')])
    vertices = [[float(s) for s in f.readline().strip().split(' ')] for ith_vertex in range(num_vertices)]
    faces = [[int(s) for s in f.readline().strip().split(' ')][1:] for ith_face in range(num_faces)]
    return vertices, faces

In [ ]:
# In case of obj files, convert to off files first
sample_file = '/content/model.off'
convert_obj_to_off('/content/model.obj', sample_file)

# Read the sample CAD file and plot the mesh.
# Works with direct off files.
# sample_file = path/"bed/train/bed_0001.off"
off_vertices, off_faces = read_cad_off(sample_file)



plotting_vertex = np.array(off_vertices).T
fig = go.Figure(data=[go.Mesh3d(x=plotting_vertex[0], y=plotting_vertex[1], z=plotting_vertex[2], color='lightpink', opacity=0.50)])
fig.show()

In [ ]:

# Same bed plot without the meshes (Points only)
fig = go.Figure(data=[go.Scatter3d(x=plotting_vertex[0], y=plotting_vertex[1], z=plotting_vertex[2],
                                   mode='markers', marker=dict(size=1))])
fig.show()

In [58]:
# Calculate area for each face
areas = np.zeros((len(off_faces)))
off_vertices = np.array(off_vertices)

def calculate_triangle_area(pt1, pt2, pt3):
  # Calculate the area of the triangle formed by the provided points.
  # Note: This area calculation is for a 3D triangle with Heron's formula
  side_a = np.linalg.norm(pt1 - pt2)
  side_b = np.linalg.norm(pt2 - pt3)
  side_c = np.linalg.norm(pt3 - pt1)
  perimeter = 0.5 * ( side_a + side_b + side_c)
  return max(perimeter * (perimeter - side_a) * (perimeter - side_b) * (perimeter - side_c), 0)**0.5


for i in range(len(areas)):
    areas[i] = calculate_triangle_area(off_vertices[off_faces[i][0]], off_vertices[off_faces[i][1]], off_vertices[off_faces[i][2]])

In [59]:
# Sample to a fixed number of points in each face.
# The probability of choosing a face is proportional to its area.
# The point distribution should be uniform for each face.
k = 1024
sampled_faces = random.choices(off_faces, weights=areas, k=k)

# Sample points on the surface of the chosen triangle
def sample_point_on_triangle(pt1, pt2, pt3):
    # barycentric coordinates on a triangle
    # https://mathworld.wolfram.com/BarycentricCoordinates.html
    # Another good reference: https://pharr.org/matt/blog/2019/02/27/triangle-sampling-1.html
    s, t = sorted([random.random(), random.random()])
    f = lambda i: s * pt1[i] + (t-s) * pt2[i] + (1-t) * pt3[i]
    return (f(0), f(1), f(2))

In [60]:
# Construct pointcloud.
pointcloud = np.zeros((k, 3))
for i in range(len(sampled_faces)):
    pointcloud[i] = (sample_point_on_triangle(off_vertices[sampled_faces[i][0]], off_vertices[sampled_faces[i][1]], off_vertices[sampled_faces[i][2]]))

print(len(sampled_faces))

# Save pointcloud to a npy file.
np_filename = '/content/gt_pointcloud_1024.npy'
np.save(np_filename, pointcloud, allow_pickle=True)

1024


In [ ]:
# Plot pointcloud.
# Load pointcloud from pickle file
np_filename = '/content/gt_pointcloud_1024.npy'
loaded_pointcloud = np.load(np_filename, allow_pickle=True)
plotting_pointcloud = np.array(loaded_pointcloud).T

fig = go.Figure(data=[go.Scatter3d(x=plotting_pointcloud[0], y=plotting_pointcloud[1], z=plotting_pointcloud[2],
                                   mode='markers', marker=dict(size=1))])


fig.update_layout(scene = dict(
                    xaxis = dict(title='',
                        showgrid=False,
                         showticklabels=False),
                    yaxis = dict(title='',
                         showgrid=False,
                         showticklabels=False),
                    zaxis = dict(title='',
                         showgrid=False,
                         showticklabels=False)),
                    width=700,
                  height= 400
                  )

fig.show()

In [ ]:
loaded_pointcloud = np.load('/content/pointcloud_1024.npy', allow_pickle=True)
plotting_pointcloud = np.array(loaded_pointcloud).T
fig = go.Figure(data=[go.Scatter3d(x=plotting_pointcloud[0], y=plotting_pointcloud[1], z=plotting_pointcloud[2],
                                   mode='markers', marker=dict(size=1))])

fig.update_layout(scene = dict(
                    xaxis = dict(title='',
                        showgrid=False,
                         showticklabels=False),
                    yaxis = dict(title='',
                         showgrid=False,
                         showticklabels=False),
                    zaxis = dict(title='',
                         showgrid=False,
                         showticklabels=False)),
                    width=700,
                  height= 400
                  )

fig.show()

In [ ]:
# Normalize pointcloud by subtracting mean and normalizing all points onta a unit sphere.
norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0) 
norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

# rotation around z-axis
theta = random.random() * 2. * math.pi # rotation angle
rot_matrix = np.array([[ math.cos(theta), -math.sin(theta),    0],
                       [ math.sin(theta),  math.cos(theta),    0],
                       [0,                             0,      1]])

rot_pointcloud = rot_matrix.dot(pointcloud.T).T

# add some noise
noise = np.random.normal(0, 0.02, (pointcloud.shape))
noisy_pointcloud = rot_pointcloud + noise


In [ ]:
print(np.linalg.norm(norm_pointcloud, axis=1))
print(noisy_pointcloud.shape)

In [ ]:
plotting_pointcloud = np.array(noisy_pointcloud).T
fig = go.Figure(data=[go.Scatter3d(x=plotting_pointcloud[0], y=plotting_pointcloud[1], z=plotting_pointcloud[2],
                                   mode='markers', marker=dict(size=1))])
fig.show()

In [ ]:
# Sample code to check some vectorization in tensorflow for a loss function
!pip install tensorflow==1.13.2

     |████████████████████████████████| 92.7MB 50kB/s 
     |████████████████████████████████| 3.2MB 53.0MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 368kB 55.4MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


'2.4.1'

In [ ]:
import tensorflow as tf
import numpy as np
tf.__version__

'1.13.2'

In [ ]:
%%time
## helper functions for k-random-octant loss
def compute_l2_norm(pcl_mat, octants):
    # pcl_mat: BATCH_SIZE x 1024 x 3 tensor
    # octants: BATCH_SIZE k x 3 tensor
    # output: BATCH_SIZE x 1024 x k matrix where each row contains l2 norm of each point against each of the k centers.
    
    ## this is inefficient for larger batches
    num_points = pcl_mat.shape[1]
    num_octants = octants.shape[1]
    distance = np.zeros((num_points, num_octants))
    num_batches = pcl_mat.shape[0]
    outputTensor = tf.zeros(shape = (1, pcl_mat.shape[1], octants.shape[1]))
    for batch in range(num_batches):
        output_batch = tf.zeros(shape = (1, pcl_mat.shape[1], 0))
        for octant in range(num_octants):
            # Tensor("ExpandDims_62:0", shape=(1, 1, 3), dtype=float32)
            # Tensor("sub_60:0", shape=(1, 1024, 3), dtype=float32)
            # Tensor("norm/Sqrt:0", shape=(1, 1024, 1), dtype=float32)
            single_octant_center = tf.expand_dims(
                tf.expand_dims(octants[batch, octant, :], 
                    axis = 0), axis = 0)
            diff = pcl_mat[batch, :, :] - single_octant_center
            norm_for_all_points = tf.norm(diff, axis = 2, keepdims = True)
            output_batch = tf.concat([output_batch, norm_for_all_points], axis = 2)
        outputTensor = tf.concat([outputTensor, output_batch], axis = 0)
    outputTensor = outputTensor[1:, :, :]
    return outputTensor

def find_min_distance_to_cluster(distance, pcl):
    # distance: 1024 x k matrix where each row contains l2 norm of each point against each of the k centers.
    # pcl: 1024 x 3 point clouds
    # output: Tuple of a 1xk matrix contains the row index of the min entry and another
    # 1 x k matrix where each entry contains the min_distance from a given pointcloud point to the
    # kth cluster.
    min_distance = tf.reduce_min(distance, axis=1, keepdims=True)
    min_distance_row_index = tf.argmin(distance, axis = 1)
    
    ## gather points for first batch
    anchor_points = tf.gather(pcl[0, :, :], min_distance_row_index[0, :])
    anchor_points = tf.expand_dims(anchor_points, axis = 0)

    ## stack the rest
    for i in range(1, pcl.shape[0]):
        anchor_points = tf.concat([anchor_points, tf.expand_dims(tf.gather(pcl[i, :, :], min_distance_row_index[i, :]), axis = 0)], axis = 0)
    return anchor_points, min_distance_row_index


CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 8.82 µs


In [ ]:
%%time
def getAnchorPoints(pcl_tensor, k):
    '''
    pcl_tensor: tensor of dim = (BS,N_pts,3)
    '''
    min_vals = tf.reduce_min(pcl_tensor, axis = 1)  
    max_vals = tf.reduce_max(pcl_tensor, axis = 1)
    intervals = (max_vals - min_vals)/k
    xyz_inits = min_vals - intervals/2

    # tf_range = tf.range(xyz_inits, max_vals, delta=intervals)

    # with tf.Session() as s:
    #   print('MIN: ', s.run(min_vals))
    #   print('MAX: ', s.run(max_vals))
    #   print('Intervals: ', s.run(intervals))
    #   print('INITS: ', s.run(xyz_inits))
    #   print('Range: ', s.run(tf_range))
    # init_vals = 
    
    ## this works but it's inefficient for large values of k
    batchArray = []
    for batch in range(pcl_tensor.shape[0]):
        
        x_min, y_min, z_min = min_vals[batch, 0], min_vals[batch, 1], min_vals[batch, 2]
        # print(x_min, y_min, z_min)
        x_max, y_max, z_max = max_vals[batch, 0], max_vals[batch, 1], max_vals[batch, 2]
        x_interval, y_interval, z_interval = intervals[batch, 0], intervals[batch, 1], intervals[batch, 2]
        ## compute geometric centers
        searchVolumes = []
        overallIndex = 1
        x_init, y_init, z_init = x_min - x_interval/2, y_min - y_interval/2, z_min - z_interval/2
        for p in range(0, k):
            x_init += x_interval
            for q in range(0, k):
                y_init += y_interval
                for r in range(0, k):
                    z_init += z_interval
                    searchVolumes.append([x_init, y_init, z_init])
                    overallIndex += 1
                z_init = z_min - z_interval/2
            y_init = y_min - y_interval/2
                
        batchArray.append(searchVolumes)
    octants = tf.convert_to_tensor(batchArray)

    ## get anchor points' distances
    ## L2_norm([1024, 3], [27, 3]) ==> [1024, 27] all point clouds dist to all geometric centers ==> [1, 27]
    distance = compute_l2_norm(pcl_tensor, octants)
    return find_min_distance_to_cluster(distance, pcl_tensor)

def getCenterPoint(pcl_tensor):
    min_vals = tf.reduce_min(pcl_tensor, axis = 1)
    max_vals = tf.reduce_max(pcl_tensor, axis = 1)
    geometric_center = tf.expand_dims(min_vals + (max_vals - min_vals)/2, axis = 1)
    l2_norms = compute_l2_norm(pcl_tensor, geometric_center)
    return find_min_distance_to_cluster(l2_norms, pcl_tensor)


CPU times: user 0 ns, sys: 10 µs, total: 10 µs
Wall time: 14.5 µs


In [ ]:
%%time
# Create a random tensor of size (BS, N, 3)
gt = tf.random.uniform(shape=(2, 1024, 3))

# with tf.Session() as s:
  # print('GT: ', s.run(gt))
getAnchorPoints(gt, k=3)
getAnchorPoints(gt, k=3)

CPU times: user 1.61 s, sys: 15.8 ms, total: 1.63 s
Wall time: 1.63 s


In [ ]:
print(gt_anchor_points)

Tensor("concat_284:0", shape=(2, 27, 3), dtype=float32)
